In [1]:
%%bash

pip install pgeocode -q
pip install folium -q
pip install numpy -q 
pip install pandas -q
pip install branca -q

In [2]:
#Libraries
import folium
import folium.plugins as plugins
import pandas as pd
import numpy as np
import pgeocode
import branca.colormap as cm
url = 'https://raw.githubusercontent.com/nychealth/coronavirus-data/master/archive/recent-4-week-by-modzcta.csv'
df = pd.read_csv(url, error_bad_lines=False)
#df.head()

/var/folders/b1/4962v1r500j8ybdm7cs4fkxr0000gn/T/ipykernel_4289/3686962168.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(url, error_bad_lines=False)


In [3]:
data = df[['MODIFIED_ZCTA','NEIGHBORHOOD_NAME','COVID_CASE_COUNT_WEEK4','COVID_CASE_COUNT_WEEK3','COVID_CASE_COUNT_WEEK2','COVID_CASE_COUNT_WEEK1']]
data = data.rename(columns={'MODIFIED_ZCTA':'zipcode','NEIGHBORHOOD_NAME':'neighborhood','COVID_CASE_COUNT_WEEK4':'week_4','COVID_CASE_COUNT_WEEK3':'week_3','COVID_CASE_COUNT_WEEK2':'week_2','COVID_CASE_COUNT_WEEK1':'week_1'})

In [4]:
#data

In [5]:
#Changing type
data.zipcode = data.zipcode.astype(str)
data.week_1 = data.week_1.astype(int)
data.week_2 = data.week_2.astype(int)
data.week_3 = data.week_3.astype(int)
data.week_4 = data.week_4.astype(int)
#Adding coordinate values
data['latitude'] = np.nan
data['longitude'] = np.nan
#Initializing the geocode data for US
usGeo = pgeocode.Nominatim('us')
#Do not forget to use loc when updating values in pandas dataframe
for index, row in data.iterrows():
    zipcode = row['zipcode']
    data.loc[index,'latitude'] = usGeo.query_postal_code(zipcode)['latitude']
    data.loc[index,'longitude'] = usGeo.query_postal_code(zipcode)['longitude']

In [6]:
#data.head()

In [7]:
#Create empty list of lists
nb_weeks = 4
dataByWeek = x = [[] for i in range(nb_weeks)]
for index, row in data.iterrows():
    dataByWeek[0].append([row['latitude'],row['longitude'],row['week_1']])
    dataByWeek[1].append([row['latitude'],row['longitude'],row['week_2']])
    dataByWeek[2].append([row['latitude'],row['longitude'],row['week_3']])
    dataByWeek[3].append([row['latitude'],row['longitude'],row['week_4']])

In [8]:
weeks_index=['Week 1','Week 2','Week 3','Week 4']

In [9]:
#Initialize map position and zoom on NYC
nycMap = folium.Map(location=[40.730610, -73.935242], default_zoom_start=10)
hm = plugins.HeatMapWithTime(dataByWeek, index=weeks_index,auto_play=False, max_opacity=0.3, use_local_extrema=True)
hm.add_to(nycMap)
nycMap

In [10]:
#nycMap.save('nyc_heatmap_timeseries.html')

In [11]:
#Example 2
color = []
#Generate color palette with branca.colormap
for i in ('week_1', 'week_2', 'week_3', 'week_4'):
    maxVal = max(data[str(i)])
    minVal = min(data[str(i)])
    color.append(cm.LinearColormap(['white', 'yellow', 'red'], vmin=minVal, vmax=maxVal))


In [12]:
#color

In [13]:
def create_geojson_features(df,color):
    features = []
    fg_markers = folium.FeatureGroup(name="Places")
    for index, row in df.iterrows():
        #Markers
        folium.Marker(
            location=[row['latitude'],row['longitude']],
            popup = ('<strong>Zipcode:</strong> ' + row['zipcode'] + '<br>'
                     '<strong>Neighborhood:</strong> ' + row['neighborhood'] + '<br>'
                    ),
            zipcode=row['zipcode']
            ).add_to(fg_markers)
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point', 
                'coordinates':[row['longitude'],row['latitude']]
            },
            'properties': {
                'time': '2020-01-24',
                'style': {'color' : 'black'},
                'icon': 'circle',
                'iconstyle':{
                    'fillColor': color[0](row['week_1']),
                    'fillOpacity': 0.8,
                    'stroke': 'true',
                    'radius': 7
                },
                 'popup': '<strong>Zipcode:</strong> ' + row['zipcode'] + '<br>'
                          '<strong>Neighborhood:</strong> ' + row['neighborhood'] + '<br>'
                          '<strong>Positive Cases:</strong> ' + str(row['week_1'])
            }
        }
        features.append(feature)
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point', 
                'coordinates':[row['longitude'],row['latitude']]
            },
            'properties': {
                'time': '2020-01-31',
                'style': {'color' : 'black'},
                'icon': 'circle',
                'iconstyle':{
                    'fillColor': color[1](row['week_2']),
                    'fillOpacity': 0.8,
                    'stroke': 'true',
                    'radius': 7
                },
                'popup': '<strong>Zipcode:</strong> ' + row['zipcode'] + '<br>'
                         '<strong>Neighborhood:</strong> ' + row['neighborhood'] + '<br>'
                         '<strong>Positive Cases:</strong> ' + str(row['week_2'])
            }
        }
        features.append(feature)
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point', 
                'coordinates':[row['longitude'],row['latitude']]
            },
            'properties': {
                'time': '2020-02-07',
                'style': {'color' : 'black'},
                'icon': 'circle',
                'iconstyle':{
                    'fillColor': color[2](row['week_3']),
                    'fillOpacity': 0.8,
                    'stroke': 'true',
                    'radius': 7
                },
                'popup': '<strong>Zipcode:</strong> ' + row['zipcode'] + '<br>'
                         '<strong>Neighborhood:</strong> ' + row['neighborhood'] + '<br>'
                         '<strong>Positive Cases:</strong> ' + str(row['week_3'])
            }
        }
        features.append(feature)
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point', 
                'coordinates':[row['longitude'],row['latitude']]
            },
            'properties': {
                'time': '2020-02-14',
                'style': {'color' : 'black'},
                'icon': 'circle',
                'iconstyle':{
                    'fillColor': color[3](row['week_4']),
                    'fillOpacity': 0.8,
                    'stroke': 'true',
                    'radius': 7
                },
                'popup': '<strong>Zipcode:</strong> ' + row['zipcode'] + '<br>'
                         '<strong>Neighborhood:</strong> ' + row['neighborhood'] + '<br>'
                         '<strong>Positive Cases:</strong> ' + str(row['week_4'])
            }
        }
        features.append(feature)
    return features, fg_markers

In [14]:
def make_map(features, fg_markers):
    Map = folium.Map(location=[40.730610, -73.935242], default_zoom_start=10)
    folium.plugins.TimestampedGeoJson(
        {'type': 'FeatureCollection',
        'features': features}
        , period='P1D'
        , add_last_point=True
        , auto_play=False
        , loop=False
        , max_speed=1
        , loop_button=True
        , date_options='YYYY-MM-DD'
        , time_slider_drag_update=True
    ).add_to(Map)
    fg_markers.add_to(Map)
    folium.LayerControl(position='topleft').add_to(Map)
    folium.plugins.Search(
        layer=fg_markers,
        search_label="zipcode",
        search_zoom=15,
        collapsed=True,
        placeholder='Search By Zipcode',
    ).add_to(Map)
    return Map

In [15]:
features, fGroup = create_geojson_features(data,color)

In [16]:
test = make_map(features,fGroup)

In [17]:
test

In [18]:
#test.save('nyc_timeseries_map.html')